<center> 
    <h1>
Linear and Quadratic Knapsack Problem
    </h1>
    <h2>
Large Scale Optimization for Data Science
    </h2>
    <h3>
Kale-ab Tessera, 1973752
    </h3>
</center>

In the Knapsack problem, our goal is to find the optimal combination of objects which are bound by a total weight W and which achieve the highest profit / value (v). The problem can be applied to various real life applications such resource allocation problems.

Two variants of the Knapsack problem are considered. The **Linear Knapsack Problem**, where items have individual weights and values, and the **Quadratic Knapsack Problem** where there is an additional term in the objective functions, which represents the extra profit gained from choosing a particular combination of items (in our case pairs). 

## Problem 1 - Linear Knapsack Problem

Linear Implementation of Knapsack Problem.
<br/> $$( v_i , w_i) = \{ (2,7),(6,3),(8,3),(7,5),(3,4),(4,7),(6,5),
(5,4),(10,15),(9,10),(8,17),(11,3),(12,6),(15,11),(6,6),
(8,14),(13,4),(14,8),(15,9),(16,10),(13,14),(14,17),(15,9),(26,24),
(13,11),(9,17),(25,12),(26,14) \}$$ with **total capacity W = 30**

### 1.1 Greedy Implementation

In [1]:
import numpy as np
def calculateScore(v,w,n):
    score = np.array([])
    for i in range(n):
        score = np.append(score,round((v[i]/w[i]),3)) 
    scoreIndexs = np.argsort(-score)
    
    return scoreIndexs

def linearKnapSack(v,w,n,W,shouldPrint=False):
    sumItemsInKnapsack = 0
        
    scoreIndexs =  calculateScore(v,w,n)
    indexofElementsInKnapsack = np.zeros(n).astype(int)
    
    for i in range(n):
       indexOfLargestElement = scoreIndexs[i]
       if((sumItemsInKnapsack+w[indexOfLargestElement]) <= W ):
           indexofElementsInKnapsack[indexOfLargestElement] = 1 
           sumItemsInKnapsack += w[indexOfLargestElement]
           if(shouldPrint):
               print("Adding (",v[indexOfLargestElement],",",w[indexOfLargestElement], 
                     ") to the knapsack")    
    
    if(shouldPrint):
        print("***************************")
        print("Arrays: ")
        print("v_i = {0}".format(v))
        print("w_i = {0}".format(w))
        print("x_i = {0}".format(indexofElementsInKnapsack))
        
    return indexofElementsInKnapsack

In [2]:
n = 28
v = np.array([2,6,8,7,3,4,6,5,10,9,8,11,12,15,6,8,13,14,15,16,13,14,15,26,13,9,25,26])
w = np.array([7,3,3,5,4,7,5,4,15,10,17,3,6,11,6,14,4,8,9,10,14,17,9,24,11,17,12,14])
W = 30

print("Problem 1 - Greedy Algorithm ")    
indexofElementsInKnapsack = linearKnapSack(v,w,n,W)
print("Selected w:",w[indexofElementsInKnapsack==1])  
print("Selected v:",v[indexofElementsInKnapsack==1])  
print("Profit - linear knapsack:" , np.sum(v[indexofElementsInKnapsack==1]))
print("Weight - linear knapsack:" , np.sum(w[indexofElementsInKnapsack==1]))

Problem 1 - Greedy Algorithm 
Selected w: [ 3  3  5  3  4 12]
Selected v: [ 6  8  7 11 13 25]
Profit - linear knapsack: 70
Weight - linear knapsack: 30


### 1.2 Polynomial Time Approximation Algorithm
<br/>The function that retrieves all subsets was implemented recursively in order to find the optimal solution.

In [3]:
def findSubsets(allSubsets,subset,currentIndex,k):      
    if(len(subset) == currentIndex):
        return allSubsets
    
    newSet = []
    for i in range(0,len(allSubsets)):
      if(len(allSubsets[i]) < k):
        newSet = allSubsets[i].copy()
        newSet.append(subset[currentIndex])
        allSubsets.append(newSet)
        
    findSubsets(allSubsets, subset, currentIndex+1,k)

def isLargerThanCapicity(chosenW,W):
      sumWeights = np.sum(chosenW)
      if(sumWeights > W):
          return True
      else:
          return False
        
def finaAllSubsets(array, subsetSize,k):
    allSubsets = [[]]
    findSubsets(allSubsets,array,0,k)
    returnSet = []
    for sets in allSubsets:
        if(len(sets) == subsetSize):
            returnSet.append(sets)      
    return returnSet

def PTAS(v,w,n,W,minSizeSubset,k,shouldPrint=False):
    currentW = 0
    sendW = 0
    
    allSubsets = [[]]
    indexOfSet = []
    for i in range(0,28):
        indexOfSet.append(i)
        
    allSubsets = finaAllSubsets(indexOfSet,minSizeSubset,k)
    allSubsetsNumpy = np.array(allSubsets)
    highestProfit = 0
    bestSetIndex = []
    bestLinearIndex = []
    for sets in allSubsetsNumpy:
        chosenV = np.take(v,sets)
        chosenW = np.take(w,sets)  
        if(isLargerThanCapicity(chosenW,W) == False):
            otherV = np.take(v,list(set(indexOfSet) - set(sets)))
            otherW = np.take(w,list(set(indexOfSet) - set(sets)))
            sumProfitPTAS = np.sum(chosenV)
            sumWeights = np.sum(chosenW)
            remainingW = W - sumWeights
            indexofElementsInKnapsack = linearKnapSack(otherV,otherW,len(otherV) ,remainingW)
            totalProfit = sumProfitPTAS + np.sum(otherV[indexofElementsInKnapsack==1])
            if(shouldPrint):
                print("Weight - PTAS: ",sumWeights)
                print("Profit - PTAS: ",sumProfitPTAS)
                print("Total Profit: ",totalProfit)
                print("***************************")
                
            if(totalProfit >highestProfit ):
                highestProfit = totalProfit
                bestSetIndex = sets
                bestLinearIndex = indexofElementsInKnapsack
                bestOtherV = otherV
                bestOtherW = otherW
    
    bestLinearIndexValues = np.where(bestLinearIndex==1)
    print("Best Indexs for PTAS:",bestSetIndex)  
    print("Selected v - PTAS:",np.take(v,bestSetIndex))  
    print("Selected v - Linear:",np.take(bestOtherV,bestLinearIndexValues)[0])  
    print("Selected v - all:",np.concatenate([np.take(v,bestSetIndex),
    np.take(bestOtherV,bestLinearIndexValues)[0]]))
    print("Selected w - all",np.concatenate([np.take(w,bestSetIndex),
    np.take(bestOtherW,bestLinearIndexValues)[0]]))
    print("Total Weight - all:",np.sum(np.concatenate([np.take(w,bestSetIndex),
    np.take(bestOtherW,bestLinearIndexValues)[0]])))
    print("Best Profit - all:",highestProfit)


In [10]:
print("Problem 1 - Polynomial Time Approximation Algorithm (PTAS) ")
minSizeSubset = 3
k = 10
PTAS(v,w,n,W,minSizeSubset,k)

Problem 1 - Polynomial Time Approximation Algorithm (PTAS) 
Best Indexs for PTAS: [ 2 11 17]
Selected v - PTAS: [ 8 11 14]
Selected v - Linear: [13 25]
Selected v - all: [ 8 11 14 13 25]
Selected w - all [ 3  3  8  4 12]
Total Weight - all: 30
Best Profit - all: 71


## Problem 2 - Quadratic Knapsack Problem

Quadratic Implementation of Knapsack Probelm.
Extra profit when choosing certain pairs are considered.
<br/> 
$$ v_i = \{7, 6, 13, 16, 5, 10, 9, 23, 18, 12, 9, 22, 17, 32, 8\} $$ <br/>
$$ w_i = \{13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26\} , W = 50 $$ <br/>
and $$ p_{ij} = \{(12, 7, 6, 13, 8, 11, 7, 15, 23, 14, 15, 17, 9, 15, 15,
13, 10, 15, 9, 10, 8, 17, 11, 13, 12, 16, 15, 11, 16, 6,
8, 14, 13, 4, 14, 8, 15, 9, 16, 10, 13, 14, 14, 17, 15, 14, 6, 24, 13, 4, 9, 7, 25, 12, 6,
6, 16, 10, 15, 14, 2, 13, 12, 16, 9, 11, 23, 10, 21, 8
18, 4, 13, 14, 14, 17, 15, 9, 16, 12, 3, 14, 14, 27, 15, 16, 13, 14, 7, 17, 28, 5, 19, 6, 18, 13, 4, 13, 16, 11,19, 13, 15, 12, 16)\}
$$

In [10]:
import random
def generateRandomNumbers(numRandomNumbers,maxRandomNumber,randomSeed ):
#     random.seed(randomSeed)
#     listRandomIndexs = []
#     while len(listRandomIndexs) < numRandomNumbers:
#         randomI = random.randint(0,maxRandomNumber-1)
#         if randomI not in listRandomIndexs: listRandomIndexs.append(randomI)        
#     return listRandomIndexs
    np.random.seed(randomSeed)
    listRandomIndexs = np.random.choice(range(15), 7, replace=False)
    return listRandomIndexs

def sortByEfficiencyFunction(arrayOfPairs,p,w):
    allEfficiencies = np.array([])
    for pair in arrayOfPairs:
        P_ij = p[pair[0]][pair[1]]
        w_i = w[pair[0]]
        w_j = w[pair[1]]
        efficiency = P_ij/(w_i+w_j)
        allEfficiencies = np.append(allEfficiencies,efficiency)
        
    efficiencyIndex = np.argsort(-allEfficiencies)
    return efficiencyIndex

def isAnElementFromPairAlreadySelected(pairToAdd,pairsAlreadyChosen):
    indexFirstElementInPair = pairToAdd[0]
    indexSecondElementInPair = pairToAdd[1]
    
    if (indexFirstElementInPair in pairsAlreadyChosen 
        or indexSecondElementInPair in pairsAlreadyChosen):
        return True
    else:
        return False
    
def allElementsToKnapSack(arrayOfPairs,efficiencyIndexs,v,w,W,p):
    k = len(efficiencyIndexs)
    sumWeightElementsKnapsack = 0
    indexChosenElements = []
    sumProfit = 0
    indexOfSet = []
    for i in range(0,15):
        indexOfSet.append(i)
        
    for i in range(k):
        currentLargestPair = arrayOfPairs[efficiencyIndexs[i]]
        indexFirstElementInPair = currentLargestPair[0]
        indexSecondElementInPair = currentLargestPair[1]
        w_i = w[indexFirstElementInPair]
        w_j = w[indexSecondElementInPair]
        if((sumWeightElementsKnapsack+w_i+w_j) <= W):
            if(isAnElementFromPairAlreadySelected(currentLargestPair,indexChosenElements) == False):
                indexChosenElements.append(indexFirstElementInPair)
                indexChosenElements.append(indexSecondElementInPair)
                otherV = np.take(v,list(set(indexOfSet) - set(currentLargestPair)))
                otherW = np.take(w,list(set(indexOfSet) - set(currentLargestPair)))
                sumWeightElementsKnapsack += (w_i+w_j)
                sumProfit += (v[indexFirstElementInPair] + v[indexSecondElementInPair])
                print("Adding (",currentLargestPair,",",w_i,w_j, ") to the knapsack")   
    
    remainingW = W - sumWeightElementsKnapsack
    
    print("Weight - PAIRS: ",sumWeightElementsKnapsack)
    print("Profit - PAIRS: ",sumProfit)
    
    indexofElementsInKnapsack = linearKnapSack(otherV,otherW,len(otherV) ,remainingW)
    print("Weight - linear knapsack:" , np.sum(otherW[indexofElementsInKnapsack==1]))
    print("Profit - linear knapsack:" , np.sum(otherV[indexofElementsInKnapsack==1]))
    
    print("Selected v - PAIRS:",np.take(v,indexChosenElements))  
    print("Selected v - Linear:",otherV[indexofElementsInKnapsack==1])  
    print("Selected w - PAIRS:",np.take(w,indexChosenElements))  
    print("Selected w - Linear:",otherW[indexofElementsInKnapsack==1]) 
    
    print("Selected v - all:",np.concatenate([np.take(v,indexChosenElements),otherV[indexofElementsInKnapsack==1]]))     
    
    print("Selected w - all: ",np.concatenate([np.take(w,indexChosenElements),
    otherW[indexofElementsInKnapsack ==1]]))
    print("Total Weight - all: ",np.sum(np.concatenate([np.take(w,indexChosenElements),
    otherW[indexofElementsInKnapsack ==1]])))
    print("Total Profit - all:",np.sum(np.concatenate([np.take(v,indexChosenElements),
    otherV[indexofElementsInKnapsack==1]])))


In [12]:
print("Problem 2 - Quadratic Knapsack Problem ")
v = np.array([7, 6, 13,16, 5, 10, 9, 23, 18, 12, 9, 22, 17, 32, 8])
w = np.array([13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
W = 50
p = np.array([
7	,12	,7	,6	,13	,8	,11	,7	,15	,23	,14	,15	,17	,9	,15,
12	,6	,15	,13	,10	,15	,9	,10	,8	,17	,11	,13	,12	,16	,15,
7	,15	,13	,11	,16	,6	,8	,14	,13	,4	,14	,8	,15	,9	,16,
6	,13	,11	,16	,10	,13	,14	,14	,17	,15	,14	,6	,24	,13	, 4,
13	,10	,16	,10	,5	,9	,7	,25	,12	,6	,6	,16	,10	,15	,14,
8	,15	,6	,13	,9	,10	,2	,13	,12	,16	,9	,11	,23	,10	,21,
11	,9	,8	,14	,7	,2	,9	,8	,18	,4	,13	,14	,14	,17	,15,
7	,10	,14	,14	,25	,13	,8	,23	,9	,16	,12	,3	,14	,14	,27,
15	,8	,13	,17	,12	,12	,18	,9	,18	,15	,16	,13	,14	,7	,17,
23	,17	,4	,15	,6	,16	,4	,16	,15	,12	,28	,5	,19	,6	,18,
14	,11	,14	,14	,6	,9	,13	,12	,16	,28	,9	,13	,4	,13	,16,
15	,13	,8	,6	,16	,11	,14	,3	,13	,5	,13	,22	,11	,19	,13,
17	,12	,15	,24	,10	,23	,14	,14	,14	,19	,4	,11	,17	,15	,12,
9	,16	,9	,13	,15	,10	,17	,14	,7	,6	,13	,19	,15	,32	,16,
15	,15	,16	,4	,14	,21	,15	,27	,17	,18	,16	,13	,12	,16	, 8
    ])
p = p.reshape(15,15)
k = 7
size = 15
randomSeed = 11

randomNumberIndex = generateRandomNumbers(k,size,randomSeed)
print("Indexs for sampled items:" ,randomNumberIndex)
arrayOfPairs = finaAllSubsets(randomNumberIndex,2,k)
print("Pairs: ",arrayOfPairs)
efficiencyIndexs = sortByEfficiencyFunction(arrayOfPairs,p,w)
scoreIndex = np.argsort(efficiencyIndexs)
allElementsToKnapSack(arrayOfPairs,scoreIndex,v,w,W,p)

Problem 2 - Quadratic Knapsack Problem 
Indexs for sampled items: [ 8  3 14  6  4  5 13]
Pairs:  [[8, 3], [8, 14], [3, 14], [8, 6], [3, 6], [14, 6], [8, 4], [3, 4], [14, 4], [6, 4], [8, 5], [3, 5], [14, 5], [6, 5], [4, 5], [8, 13], [3, 13], [14, 13], [6, 13], [4, 13], [5, 13]]
Adding ( [8, 3] , 13 15 ) to the knapsack
Adding ( [5, 13] , 9 12 ) to the knapsack
Weight - PAIRS:  49
Profit - PAIRS:  76
Weight - linear knapsack: 0
Profit - linear knapsack: 0
Selected v - PAIRS: [18 16 10 32]
Selected v - Linear: []
Selected w - PAIRS: [13 15  9 12]
Selected w - Linear: []
Selected v - all: [18 16 10 32]
Selected w - all:  [13 15  9 12]
Total Weight - all:  49
Total Profit - all: 76


In [7]:
import numpy as np
np.random.seed(24)
np.random.choice(range(15), 7, replace=False)


array([ 5, 12,  8, 14,  6, 11, 13])

In [13]:
for i in range(15):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
